# 네이버 맛집 크롤링


## 크롤링 사이트 

https://store.naver.com/restaurants/list?bounds=127.0189674%3B37.4931153%3B127.0461162%3B37.5044717&query=%EB%A7%9B%EC%A7%91&sessionid=Jb0idF7A9JvBaVT%2BwKa0xBHv

## 방법 

1. 관련도순
2. 클릭전 페이지 1개 항목 크롤링
3. 클릭후 모든 정보 크롤링
4. 다음 물건 크롤링
5. 반복후 페이지 넘기기

In [4]:
# selenium 설치
# !pip install selenium

In [1]:
import bs4
import requests
import time
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import re
import requests as rq
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')           # 성가신 warning은 off 해준다. 

import os


In [2]:
# 웹 드라이버 가동 (네이버 플레이스)
driver = webdriver.Chrome('chromedriver')
url = 'https://store.naver.com/restaurants/list?query=%EB%A7%9B%EC%A7%91&sessionid=Jb0idF7A9JvBaVT%2BwKa0xBHv&sortingOrder=precision'
driver.get(url)

In [141]:
# 파일 최초 저장여부
chk = False

# IP차단 방지를 위한 딜레이
delay_time = 1

#(중간에 끊겼을때..)크롤링 시작페이지 지정
start_page = 1

page = ['a:nth-child(3)', 'a:nth-child(4)', 'a:nth-child(5)', 'a:nth-child(6)', 'a.btn_direction.btn_next']
pagenumber = 0

total_res_count = round(867386 / (20*5))

In [4]:
def grepindex(reg_exp,my_text):
    result = []
    index_result = []
    reg_exp_compiled = re.compile(reg_exp)
    for i in range(len(my_text)):
        if reg_exp_compiled.search(my_text[i]):
            result.append(my_text[i])
            index_result.append(i)
    return index_result

## 크롤링

In [143]:
for i in range(total_res_count):
    for p in page:
        pagenumber += 1
        print(pagenumber, ": 페이지 시작")
        # 딕셔너리 초기화
        data_dic = {
            'Store_name': [],
            'Category': [],
            'Review': [],
            'TV': [],
            'Biztel': [],
            'Address': [],
            'Menu': [],
            'Homepage': [],
            '페이스북': [],
            '인스타그램': [],
            '블로그': [],
            '웹사이트': [],
            'Convenience': [],
            'Desc': [],
        }


        # 1개 클릭
        for i in range(20):
            time.sleep(1)
            # 가게 정보 밖에서 가져올 수 있는 부분
            palce_info_outside = driver.find_elements_by_css_selector('#container > div.placemap_area > div.list_wrapper > div > div.list_area > ul > li')[i].text.split('\n')
            # 가게 이름
            index = grepindex("^[A-T] ", palce_info_outside)[0]
            store_name = re.sub('^[A-T] ', '', palce_info_outside[index])
            # 리뷰수
            review = palce_info_outside[-1].split('10만원')[0].split('만원')[0].split('리뷰 ')[-1]
            place = driver.find_elements_by_css_selector('#container > div.placemap_area > div.list_wrapper > div > div.list_area > ul > li')[i]
            category = place.find_element_by_css_selector('div > div > div.tit > span > span').text
            # TV출연 했으면
            try:
                tv = place.find_elements_by_css_selector(' div > div > div.txt.tv.ellp')[0].text
            # 안했으면
            except:
                tv = ''
            # 클릭
            place.find_elements_by_css_selector('div > div > div.tit > span > a > span')[0].click()
            # 활성창 변경
            driver.switch_to_window(driver.window_handles[-1])

            # 디테일 페이지 정보 수집
            biztel = driver.find_element_by_css_selector('div.list_item_biztel').text
            address = driver.find_element_by_css_selector('div.list_item_address').text
            menu = driver.find_element_by_css_selector('div.list_item_menu').text

            try:
                homepage = driver.find_element_by_css_selector('div.list_item_homepage > div > a').text
            except NoSuchElementException:
                homepage = ''

            # sns 정보 수집
            colectsnslist = ['페이스북', '인스타그램', '블로그', '웹사이트']
            sns = driver.find_elements_by_css_selector('div.list_item_homepage > div > ul > li > a.link')
            snslist = []
            for a in sns:
                snslist.append(a.text)

            for c in colectsnslist:
                if c in snslist:
                    for s in sns:
                        if s.text == c:
                            i = s
                    data_dic[c].append(i.get_attribute('href'))
                else:
                    data_dic[c].append('')

            try:
                convenience = driver.find_element_by_css_selector('div.list_item_convenience').text
            except NoSuchElementException:
                convenience = ''
            try:
                try:
                    btn_more = driver.find_element_by_css_selector('#content > div:nth-child(2) > div > div > div.list_item.list_item_desc > div > div > a')
                    btn_more.click()
                    desc = driver.find_element_by_css_selector('div.list_item_desc').text
                except NoSuchElementException:
                    desc = driver.find_element_by_css_selector('div.list_item_desc').text
            except NoSuchElementException:
                desc = ''

            data_dic['Biztel'].append(biztel)
            data_dic['Address'].append(address)
            data_dic['Menu'].append(menu)
            data_dic['Homepage'].append(homepage)

            data_dic['Convenience'].append(convenience)
            data_dic['Desc'].append(desc)

            data_dic['Store_name'].append(store_name)
            data_dic['Review'].append(review)
            data_dic['Category'].append(category)
            data_dic['TV'].append(tv)

            driver.close()
            driver.switch_to_window(driver.window_handles[-1])

        # 페이지별 데이터 프레임 저장, excel파일에 저장
        df = pd.DataFrame.from_dict(data_dic)
        if chk == False:
            chk = True
            df.to_csv('네이버_맛집_크롤링', index=False, encoding='utf-8-sig')
        else:
            df.to_csv(title, index=False, encoding='utf-8-sig', mode='a', header=False )

        # 다음페이지로 넘기기
        nextpage = driver.find_element_by_css_selector(f'#container > div.placemap_area > div.list_wrapper > div > div.list_area > div > div.pagination_inner > {p}')
        nextpage.click()
        time.sleep(delay_time)

1 : 페이지 시작
2 : 페이지 시작


KeyboardInterrupt: 

In [130]:
data_dic

{'Store_name': ['베이징코야 역삼동점중식당',
  '스타벅스 역삼역점카페',
  '스시산원 청초밥,롤',
  '모터시티양식',
  '루프탑 클라우드',
  '송추가마골 인어반 강남점',
  '그릴640육류,고기요리',
  '롱브레드 베이커리샌드위치',
  '어육장해물,생선요리',
  '동경전통육개장한식',
  '삼다연돼지고기구이',
  '바비레드 강남양식',
  '크레이지후라이 역삼점',
  '현대기사식당기사식당',
  '바게트케이베이커리',
  '교센',
  '연안식당 역삼점해물,생선요리',
  '로즈힐 역삼점육류,고기요리',
  '순남시래기 역삼점백반,가정식',
  '공차 역삼점'],
 'Category': ['중식당',
  '카페',
  '초밥,롤',
  '양식',
  '',
  '',
  '육류,고기요리',
  '샌드위치',
  '해물,생선요리',
  '한식',
  '돼지고기구이',
  '양식',
  '',
  '기사식당',
  '베이커리',
  '',
  '해물,생선요리',
  '육류,고기요리',
  '백반,가정식',
  ''],
 'Review': ['151',
  '104',
  '175',
  '542',
  '767',
  '1045',
  '71',
  '1541',
  '2302',
  '142',
  '94',
  '2,9572',
  '702',
  '157',
  '1761',
  '2957',
  '612',
  '34',
  '169',
  '311'],
 'TV': ['생생정보통',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '2TV생생정보',
  '2TV생생정보',
  '',
  '생방송오늘저녁',
  '',
  '전지적참견시점',
  'MBC파워매거진',
  '',
  '',
  '',
  '',
  ''],
 'Biztel': ['02-733-5292',
  '02-758-8314',
  '02-538-5550',
  '02-2051-8877',
  '02-2050-6035',

In [131]:
# 한페이지당 딕셔너리에 담긴 자료 갯수 (오류 찾기)
a = data_dic.values()
for b in a:
    print(len(b))

20
20
20
20
20
20
20
20
20
20
20
20
20
20


In [121]:
driver.close()

In [126]:
driver.switch_to_window(driver.window_handles[-1])

In [127]:
driver.current_url

'https://store.naver.com/restaurants/list?page=2&query=%EB%A7%9B%EC%A7%91&sessionid=Jb0idF7A9JvBaVT%2BwKa0xBHv&sortingOrder=precision'